# C.C.A.S.A.: Car Concentration And Security Assistant
Bracco Filippo & Di Vece Chiara 


## Introduction

Stanchezza, nervosismo, distrazione e sonnolenza sono le cause di oltre il 66% degli incidenti. Questi dati allarmanti ci hanno indotto a pensare un dispositivo che possa evitare incidenti di questa natura, verificando oggettivamente che il conducente sia sempre concentrato sulla strada. Lo scopo del programma è quello di rilevare particolari condizioni  non compatibili con una guida sicura attraverso il rilevamento dei tratti del viso e, in particolar modo, degli occhi, accertandosi che lo sguardo sia rivolto verso la strada mentre il veicolo è in movimento.
In seguito risponderà adeguatamente per richiamare all’attenzione il guidatore con avvisi acustici e visivi. 
Lo scopo principale è quello di evitare molti incidenti e poter salvare molte vite. 

Fatigue, nervousness, distraction and drowsiness are the causes of more than 66% of accidents. These alarming figures prompted us to devise a device that could prevent accidents of this nature by objectively verifying that the driver is always focused on the road. The aim of the programme is to detect particular conditions that are not compatible with safe driving by detecting facial features and, in particular, the eyes, making sure that the eyes are on the road while the vehicle is moving.
It will then respond appropriately to draw the driver's attention with audible and visual warnings.
The main aim is to avoid many accidents and save many lives.

## Code

Si importano le librerie:
* **CV2**: libreria open-source per la gestione di immagini e video;
* **NUMPY**: libreria per calcolo scientifico;
* **PYGLET**: libreria per la gestione di immagini, video e suoni (utilizzata in questo codice solo per questi ultimi).

The libraries are imported:
* **CV2**: open-source library for image and video management;
* **NUMPY**: library for scientific calculation;
**PYGLET**: library for the management of images, videos and sounds (used in this code only for the latter).
*

In [ ]:
import cv2
import numpy as np
import pyglet

## Definizione costanti
**MISURAZIONI** e **SOGLIA** sono utilizzate per discriminare lo stato di *occhi aperti* oppure *occhi chiusi*.

## Constant definitions
**MEASUREMETS** and **SHEEP** are used to discriminate the state of *eyes open* or *eyes closed*.

In [ ]:
MEASUREMETS = 10
THRESHOLD = 10

## Variabili globali

**stato_occhi**: è un dizionario formato da:
* la lista *'rilevamenti'*, contenente il numero di occhi rilevati nelle ultime misurazioni (la cardinalità della lista è definita dalla costante *MISURAZIONI*), iniazialmente la lista è riempita di 2 (condizione normale);
* la variabile booleana *'occhi_chiusi'* che descrive lo stato corrente degli occhi. 

**eye_status**: is a dictionary consisting of:
* the list *'detections'*, containing the number of eyes detected in the last measurements (the cardinality of the list is defined by the constant *MEASUREMENTS*), initially the list is filled with 2 (normal condition);
* the boolean variable *'eyes_closed'* which describes the current state of the eyes.

In [ ]:
eye_status = {'detections': [2]*MEASUREMETS, 'closed_eyes': False}

## Function definition

1) La funzione **check** ha come parametro in ingresso un intero **occhi_trovati** che rappresenta il numero di occhi rilevati in un singolo frame. Lo scopo è quello di aggiornare la variabile globale *stato_occhi*: 
* viene aggiunto il valore acquisito in coda alla lista *'rilevamenti'* del dizionario;
* viene rimosso il primo elemento della suddetta lista (ovvero la rilevazione più vecchia tra quelle salvate);
* viene calcolata la somma dei rilevamenti nella lista;
* se il numero è superiore alla soglia definita, gli occhi sono aperti, quindi si assegna alla variabile *'occhi_chiusi'* del dizionario il valore *False*, *True* altrimenti.

1) The **check** function has as input parameter an integer **eyes_found** representing the number of eyes detected in a single frame. The purpose is to update the global variable *eye_state*:
* the value acquired at the end of the *'detections'* dictionary list is added;
* the first element of this list is removed (i.e. the oldest detection among the saved ones);
* the sum of the readings in the list is calculated;
* if the number is higher than the defined threshold, the eyes are opened, then the dictionary variable *'eyes_closed'* is assigned the value *False*, *True* otherwise.

In [ ]:
def check(eyes_found):
    
    eye_status['detections'].append(eyes_found)
    eye_status['detections'].pop(0)
    eyes_num = sum(eye_status['detections'])
    
    if eyes_num > THRESHOLD:
        eye_status['closed_eyes'] = False
    else:
        eye_status['closed_eyes'] = True

2) La funzione **cambia_colore** serve a cambiare il colore di pixel specifici di un'immagine. I parametri di ingresso sono l'immagine **immagine**, una lista, **pixels**, contenente le coppie di coordinate relative ai pixel il cui valore deve essere cambiato, e tre interi, **B**, **G** e **R**, che rappresentano il valore da assegnare ai pixel dell'immagine contenuti nella lista.

2) The **change_color** function is used to change the colour of specific pixels in an image. The input parameters are the **image**, a list, **pixels**, containing the pairs of coordinates relating to the pixels whose value is to be changed, and three integers, **B**, **G** and **R**, representing the value to be assigned to the image pixels contained in the list.

In [ ]:
def change_color(img, pixels, B, G, R):
    for (w,k) in pixels:
        img[w][k] = [B, G, R]

3) La funzione **draw** serve a sovrascrivere un'immagine su un'altra più grande a partire da una precisa posizione. Essa riceve in ingresso l'immagine base **img1**, la seconda immagine **img2** e due interi, **i** e **j**, che rappresentano la coordinata dell'immagine base dalla quale iniziare a copiare la seconda.

3) The **draw** function is used to overwrite an image on a larger one from a specific position. It receives as input the base image **img1**, the second image **img2** and two integers, **i** and **j**, which are the coordinates of the base image from which to start copying the second one.

In [ ]:
def draw(img1, img2, i, j):          #i = coordinata verticale, j = orizzontale
    h,w, _ = img2.shape
    img1[i:i+h, j:j+w] = img2[:h, :w] 

4) La funzione **main_face** ha come parametro in ingresso la lista **facce**. Essa contiene uno o più elementi, ognuno dei quali rappresenta una faccia rilevata nell'immagine in ingresso dalla camera; ogni elemento è una lista di quattro valori che caratterizzano la regione dell'immagine in cui è presente il volto, ovvero posizione orizzontale, posizione verticale, larghezza e altezza. La funzione restituisce i quattro parametri (**fx**, **fy**, **fw** e **fh**) della faccia con le dimensioni maggiori tra quelle contenute nella nella lista. 

4) The **main_face** function has as input parameter the **face** list. It contains one or more elements, each of which represents a face detected in the incoming image from the camera; each element is a list of four values that characterise the region of the image in which the face is present, i.e. horizontal position, vertical position, width and height. The function returns the four parameters (**fx**, **fy**, **fw** and **fh**) of the face with the largest dimensions among those contained in the list.

In [ ]:
def main_face(faces):                # restituisce i parametri della faccia più estesa rilevata nell'immagine
    fx = faces[0][0]
    fy = faces[0][1]
    fw = faces[0][2]
    fh = faces[0][3]
    
    for [x, y, w, h] in faces:
        if w*h > fw*fh:
            fx = x
            fy = y
            fw = w
            fh = h
    return fx, fy, fw, fh

## Data loading

Si caricano i file relativi alle funzioni di classificazione per faccia e occhi (materiale OpenCV).

The files relating to the classification functions for face and eyes (OpenCV material) are loaded.

In [ ]:
face_cascade = cv2.CascadeClassifier('../Assets/lib/face.xml')
eye_cascade = cv2.CascadeClassifier('../Assets/lib/glasses.xml')

Si utilizza la funzione di **cv2** per consentire l'accesso alla web cam e si setta le dimensioni dell'immagine in ingresso a 320x240 pixels.

You use the **cv2** function to allow access to the web cam and set the input image size to 320x240 pixels.

In [ ]:
cv2.VideoCapture(0)
cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

Si importano le immagini e i suoni utilizzati nel programma. 

The images and sounds used in the program are imported.

In [ ]:
box = cv2.imread('../Assets/Img/quadro.png')
start = cv2.imread('../Assets/Img/start.png')
safe_drive = cv2.imread('../Assets/Img/guida_sicura.png')
distracted_drive = cv2.imread('../Assets/Img/distrazione.png')
danger = cv2.imread('../Assets/Img/Pericolo.png')
triangle = cv2.imread('../Assets/Img/Allarme_sonno.png')
yellow_triangle = cv2.imread('../Assets/Img/attenzione_giallo.png')
open_eyes = cv2.imread('../Assets/Img/occhi_aperti.png')
sound = pyglet.media.load('../Assets/audio/alarm.mp3')
sound2 = pyglet.media.load('../Assets/audio/alarm2.mp3')

L'immagine di base sulla quale vengono sovrascritte le altre secondarie è quella del quadro:

The basic image on which the other sub-images are superimposed is that of the painting:

![Quadro](../Assets/Img/quadro.png)

Si carica un file di testo contenente coppie di numeri interi che rappresentano le posizioni dei pixel del quadro da cambiare all'accensione.

A text file is loaded containing pairs of integers representing the positions of the picture pixels to be changed at switch-on.

In [ ]:
pixels = np.loadtxt('../Assets/files/pixels_quadro.txt', dtype = 'int')

Per creare il file si sono salvate le posizioni dei pixel del quadro con valore (70, 60, 60), che rappresenta il colore del quadro spento, in una lista, la quale è stata poi salvata su file.

To create the file, the positions of the pixels of the frame with a value (70, 60), representing the colour of the off frame, were saved in a list, which was then saved to file.

![File_pixels](../Assets/Img/file_img.png)

## Initialization

Viene stampata sul quadro l'immagine *start* in posizione (100, 370) dell'immagine *quadro* tramite la funzione **draw**:

The *start* image at position (100, 370) of the *frame* image is printed on the screen using the **draw** function:

In [ ]:
draw(box, start, 100, 370)

![Start](../Assets/Img/start.png)

Si utilizzano le funzioni di **Pyglet** per la gestione degli audio: vengono creati due oggetti audio, **avviso** e  **avviso2**, che possono essere avviati e messi in pausa nel corpo del programma.

The **Pyglet** functions are used to manage audio: two audio objects, **warning** and **warning2**, are created and can be started and paused in the body of the programme.

In [ ]:
looper = pyglet.media.SourceGroup(sound.audio_format, None)          #crea looper audio (occhi chiusi)
looper.loop = True
looper.queue(sound)
warning = pyglet.media.Player()
warning.queue(looper)

looper2 = pyglet.media.SourceGroup(sound2.audio_format, None)          #crea looper audio 2 (distrazione)
looper2.loop = True
looper2.queue(sound2)
warning2 = pyglet.media.Player()
warning2.queue(looper2)

## Main code

Inizio di un ciclo **while** che stampa a schermo (tramite apposita funzione dell'ambiente) l'immagine **quadro** (*spento*) finchè su tastiera non viene premuto il tasto '**s**'. Premuto il tasto viene inserita nel quadro l'immagine **guida sicura** e viene cambiato il colore dei pixel del quadro (accensione). ![Guida sicura](../Assets/Img/guida_sicura.png)

Start of a **while** cycle that prints the **panel** image (*off*) on the screen (using a special function of the environment) until the '**s**' key is pressed on the keyboard. When the key is pressed, the **safe guide** image is inserted into the screen and the colour of the screen pixels is changed (power on). safe_guide](../Assets/Img/safe_guide.png)

In [ ]:
while cv2.waitKey(30) != ord('s'):
    cv2.imshow('Car Concentration And Security Assistant', box)
else:
    draw(box, safe_drive, 100, 370)
    change_color(box, pixels, 150, 210, 190)

In questa porzione di codice si entra in un nuovo ciclo **while** finchè non si preme su tastiera il tasto '**q**', le operazioni svolte sono:
* si acquisisce l'immagine dalla webcam tramite la funzione **read()**;
* si applica il filtro **BRG2GRAY** della libreria **CV2** all'immagine acquisita per poter utilizzare i classificatori;
* si utilizza il classificatore della faccia, il quale restituisce una lista pari al numero di facce trovate e ogni elemento di questa contiene un'ulteriore lista di quattro elementi: posizione verticale, posizione orizzontale, larghezza e altezza della regione di interesse;
* se la lista **faccia** non è vuota si cercano gli attributi della faccia rilevata più estesa (tramite la funzione **main_face** definita precedemtemente) e si riquadra l'area contenente tale volto con l'apposita funzione di **cv2**;
* all'interno della sola regione di interesse (dove è presente la faccia) si procede alla ricerca degli occhi;
* si invoca la funzione **check** per aggiornare lo stato degli occhi passandole in ingresso il numero di occhi rilevati;
* ogni occhio viene riquadrato;
* sul quadro vengono sovrascritte le immagini che segnalano la *guida sicura* e gli *occhi aperti* e si mette in pausa l'audio relativo alla distrazione **avviso2**

In this portion of the code we enter a new **while** cycle until the '**q**' key is pressed on the keyboard, the operations carried out are:
* the image from the webcam is acquired using the **read()** function;
* apply the **BRG2GRAY** filter from the **CV2** library to the acquired image in order to use the classifiers;
* use the face classifier, which returns a list equal to the number of faces found and each element of this list contains a further list of four elements: vertical position, horizontal position, width and height of the region of interest;
* if the **face** list is not empty, the attributes of the largest detected face are searched for (by means of the **main_face** function defined above) and the area containing that face is re-squared with the appropriate **cv2** function;
* within the region of interest only (where the face is present), search for the eyes;
* the **check** function is invoked to update the eyes status by passing the number of eyes detected as input;
* each eye is squared;
* the picture is overwritten with images indicating *safe driving* and *eyes open* and the distraction audio is paused **warning2**.

![open_eyes](../Assets/Img/occhi_aperti.png)

* se, invece, la lista **faccia** è vuota viene eseguito solo il ramo **else** e si si genera un avviso di distrazione visivo (immagini sul quadro) e sonoro (si avvia audio):

* if, on the other hand, the **face** list is empty, only the **else** branch is executed and a visual (images on the screen) and audio (audio starts) distraction warning is generated:
*
![Distraction](../Assets/Img/distrazione.png)

![Yellow_triangle](../Assets/Img/attenzione_giallo.png)


In [ ]:
while cv2.waitKey(30) != ord('q'):
    ret, camera = cap.read()
        
    gray = cv2.cvtColor(camera, cv2.COLOR_BGR2GRAY)
    face = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    
    if len(face) != 0:
        fx, fy, fw, fh = main_face(face)
        
        cv2.rectangle(camera,(fx,fy),(fx+fw,fy+fh),(150,210,190), 1)   
        
        roi_gray = gray[fy:fy+fh, fx:fx+fw]
        roi_color = camera[fy:fy+fh, fx:fx+fw]
        
        eyes = eye_cascade.detectMultiScale(roi_gray, maxSize=(int(fw/5),int(fh/5)), minSize=(int(fw/7),int(fh/7)))
        
        check(len(eyes))
        
        for [ex,ey,ew,eh] in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh), (150,210,190), 1)
        
        warning2.pause()
        draw(box, safe_drive, 100, 370)
        draw(box, open_eyes, 500, 455)
        
    else:
        draw(box, distracted_drive, 100, 370)
        draw(box, yellow_triangle, 500, 455)
        warning2.play()

A seguire si verifica lo stato occhi e si aggiorna opportunamente l'output:
* se nel dizionario **'stato_occhi'** la variabile **occhi_chiusi** è Vera si genera un avviso adeguato visivo (immagini sul quadro) e sonoro (si avvia audio):

Then the eye status is checked and the output is updated accordingly:

* if in the **'eye_status'** dictionary the variable **eyes_closed** is True an appropriate visual (images on the picture) and sound (audio starts) warning is generated:
*
![sleep](../Assets/Img/Allarme_sonno.png) ![danger](../Assets/Img/Pericolo.png)

altrimenti viene messo in pausa l'audio relativo agli occhi chiusi (**avviso**).

otherwise, the audio related to the closed eyes is paused (**warning**).

In [ ]:
    if eye_status['closed_eyes']:
        warning.play()
        draw(box, danger, 100, 370)
        draw(box, triangle, 500, 455)
    else:
        warning.pause()

Alla fine del ciclo si sovrappone all'immagine del quadro quella ricevuta in input dalla camera, con faccia e occhi eventualmente riquadrati, e viene mostrato a video l'immagine **quadro**.

At the end of the cycle, the image received as input from the camera is superimposed on the image of the painting, with the face and eyes framed if necessary, and the **frame** image is shown on the screen.

In [ ]:
    draw(box, camera, 235, 375)
    cv2.imshow('Car Concentration And Security Assistant', box)

## Termination

Al termine del ciclo precedente, avvenuto premendo da tastiera il tasto '**q**', vengono messi in pausa gli avvisi, nel caso in cui fossero in fase di *play*, e chiuse tutte le finestre aperte.

At the end of the previous cycle by pressing the '**q**' key on the keyboard, the alerts are paused if they are being *played* and all open windows are closed.

In [ ]:
warning.pause()
warning2.pause()
        
cap.release()
cv2.destroyAllWindows()

## Sources

* Website [OpenCV](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html) per la consultazione della documentazione sulle funzioni della libreria *cv2*;
* Download of [classificators](https://github.com/opencv/opencv/tree/master/data/haarcascades)
* Cascade algorithms operation for the [Face Detection](http://docs.opencv.org/master/d7/d8b/tutorial_py_face_detection.html#gsc.tab=0)